In [20]:
import os

In [21]:
%pwd

'c:\\Users\\Niranjan kumar\\mldemo\\research'

In [22]:
os.chdir("../")

In [23]:
%pwd

'c:\\Users\\Niranjan kumar\\mldemo'

In [24]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [25]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [27]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-09 22:50:49,899: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-09 22:50:49,939: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-09 22:50:49,969: INFO: common: yaml file: schema.yaml loaded successfully]


[2024-03-09 22:50:49,974: INFO: common: created directory at: artifacts]
[2024-03-09 22:50:49,979: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-09 22:50:52,844: INFO: 3447005916: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 49767
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7158afa18d3841c3a1285ebe7788cc1b0bae17865c7df93a1009b16d530f3362"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2A38:80FA2:1376DF4:15D67DA:65ED2E1F
Accept-Ranges: bytes
Date: Sun, 10 Mar 2024 03:50:55 GMT
Via: 1.1 varnish
X-Served-By: cache-ewr18170-EWR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1710042656.681171,VS0,VE63
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-